In [ ]:
import numpy as np
import mesa
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from scipy.stats import zscore

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

%load_ext autoreload
%autoreload 2

In [ ]:
# make the binary feature matrix
grid_size = 20
s = np.zeros((grid_size, grid_size))

# set seed
np.random.seed(0)

# add some random 1s
n_other_agents = 5
for i in range(n_other_agents):
    s[np.random.randint(2, grid_size-2), np.random.randint(2, grid_size-2)] = 1


In [ ]:
def plot_belief_style(_ax):
    # add border
    for _, spine in _ax.spines.items():
        spine.set_visible(True)
        spine.set_color('black')
        spine.set_linewidth(1)
    # ax.xaxis.set_ticks_position('top')
    # remove ticks
    _ax.set_xticks([])
    _ax.set_yticks([])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
cmap = 'binary'
sns.heatmap(s, ax=ax, cmap=cmap, cbar=False, square=True, vmin=0, vmax=1)
plot_belief_style(ax)
plt.show()


In [ ]:
X = np.meshgrid(range(grid_size), range(grid_size))
X = np.array(X).reshape(2, -1).T
y = s.flatten()
# to reconstruct s
# s_reconstructed = y.reshape(50, 50)

In [ ]:
%%time
gpc = GaussianProcessClassifier(kernel=RBF(3), random_state=0, optimizer=None).fit(X, y)
s_prob = gpc.predict_proba(X)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
cmap = 'viridis'
sns.heatmap(s_prob[:, 1].reshape(grid_size, grid_size), ax=ax, cmap=cmap, cbxar=False, square=True)
plot_belief_style(ax)
plt.show()

In [ ]:
gpc.classes_

In [ ]:
# Enable Float64 for more stable matrix inversions.
from jax import config

config.update("jax_enable_x64", True)

from time import time
import blackjax
import jax
import jax.numpy as jnp
import jax.random as jr
import jax.scipy as jsp
import jax.tree_util as jtu
from jaxtyping import (
    Array,
    Float,
    install_import_hook,
)
import matplotlib.pyplot as plt
import optax as ox
import tensorflow_probability.substrates.jax as tfp
from tqdm import trange

with install_import_hook("gpjax", "beartype.beartype"):
    import gpjax as gpx

tfd = tfp.distributions
identity_matrix = jnp.eye
key = jr.key(123)
plt.style.use(
    "https://raw.githubusercontent.com/JaxGaussianProcesses/GPJax/main/docs/examples/gpjax.mplstyle"
)


import cola
from gpjax.lower_cholesky import lower_cholesky

In [ ]:
D = gpx.Dataset(X=X, y=y)
kernel = gpx.kernels.RBF()
meanf = gpx.mean_functions.Constant()
prior = gpx.gps.Prior(mean_function=meanf, kernel=kernel)
likelihood = gpx.likelihoods.Bernoulli(num_datapoints=grid_size**2)

In [ ]:
posterior = prior * likelihood
print(type(posterior))

In [ ]:
negative_lpd = jax.jit(gpx.objectives.LogPosteriorDensity(negative=True))

optimiser = ox.adam(learning_rate=0.01)

opt_posterior, history = gpx.fit(
    model=posterior,
    objective=negative_lpd,
    train_data=y,
    optim=ox.adamw(learning_rate=0.01),
    num_iters=1000,
    key=key,
)

In [ ]:




def construct_laplace(test_inputs: Float[Array, "N D"]) -> tfd.MultivariateNormalTriL:
    map_latent_dist = opt_posterior.predict(xtest, train_data=D)

    Kxt = opt_posterior.prior.kernel.cross_covariance(x, test_inputs)
    Kxx = opt_posterior.prior.kernel.gram(x)
    Kxx += identity_matrix(D.n) * jitter
    Kxx = cola.PSD(Kxx)

    # Kxx⁻¹ Kxt
    Kxx_inv_Kxt = cola.solve(Kxx, Kxt)

    # Ktx Kxx⁻¹[ H⁻¹ ] Kxx⁻¹ Kxt
    laplace_cov_term = jnp.matmul(jnp.matmul(Kxx_inv_Kxt.T, H_inv), Kxx_inv_Kxt)

    mean = map_latent_dist.mean()
    covariance = map_latent_dist.covariance() + laplace_cov_term
    L = jnp.linalg.cholesky(covariance)
    return tfd.MultivariateNormalTriL(jnp.atleast_1d(mean.squeeze()), L)